### 2. Text-To-Cypher 활용

In [1]:
import os
from dotenv import load_dotenv
from langchain_neo4j import Neo4jGraph

load_dotenv()

#LangChain 도구 활용 - DB연결 객체 초기화
graph = Neo4jGraph(
    url = os.getenv("NEO4J_URI"),
    username = os.getenv("NEO4J_USERNAME"),
    password = os.getenv("NEO4J_PASSWORD"),
    enhanced_schema=True, # 확장 스키마 출력 설정 (그래프의 노드/관계 타입뿐만 아니라 속성 정보와 데이터 샘플까지 추출)
                          # 사용 목적 : LLM이 Cypher 쿼리를 생성할 때 정확도를 높이기 위함
)

In [2]:
# 테스트 쿼리 실행
cypher_query="""
MATCH (n:Movie)
RETURN COUNT(n) AS Movie_Count
"""

graph.query(cypher_query)

[{'Movie_Count': 4803}]

In [3]:
graph.refresh_schema()
print(graph.schema)

Node properties:
- **Movie**
  - `id`: STRING Example: "movie-4592"
  - `title`: STRING Example: "Intolerance"
  - `released`: STRING Example: "1916-09-04"
  - `overview`: STRING Example: "The story of a poor young woman, separated by prej"
  - `rating`: FLOAT Min: 0.0, Max: 10.0
  - `runtime`: INTEGER Min: 0, Max: 338
  - `tagline`: STRING Example: "The Cruel Hand of Intolerance"
- **Person**
  - `id`: STRING Example: "person-D.W. Griffith"
  - `name`: STRING Example: "D.W. Griffith"
- **Genre**
  - `id`: STRING Example: "genre-Drama"
  - `name`: STRING Example: "Drama"
Relationship properties:

The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:ACTED_IN]->(:Movie)
(:Person)-[:DIRECTED]->(:Movie)


### 2.2 LangChain을 활용한 Text-To-Cypher 구현

#### 1) GraphCyperQAChain 설정

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_neo4j import GraphCypherQAChain

#LLM 및 그래프 객체 초기화
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=0)

#자연어 질의를 Cypher 쿼리로 변환하고 실행하는 체인
cypher_chain = GraphCypherQAChain.from_llm(
    llm = llm,
    graph=graph,
    allow_dangerous_requests=True, #잠재적으로 위험한 쿼리도 허용
    verbose=True, # 중간 과정과 디버깅 정보를 상세히 출력
)

#### 2) 기본 영화 쿼리

In [6]:
#영화 제목으로 정보 검색
answer = cypher_chain.invoke({"query":"영화 'Up'에 대한 정보를 알려주세요."})

print(answer)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: 'Up'})
RETURN m
Full Context:
[{'m': {'overview': 'Carl Fredricksen spent his entire life dreaming of exploring the globe and experiencing life to its fullest. But at age 78, life seems to have passed him by, until a twist of fate (and a persistent 8-year old Wilderness Explorer named Russell) gives him a new lease on life.', 'rating': 7.7, 'runtime': 96, 'content_embedding': [-0.009156061336398125, -0.005525195971131325, -0.06422419100999832, 0.07174196094274521, 0.04694901779294014, 0.04501986876130104, 0.054215434938669205, 0.0634118989109993, 0.023096565157175064, 0.05355990678071976, -0.0645899549126625, -0.014962747693061829, 0.07053773105144501, 0.028870968148112297, -0.010752159170806408, -0.05300431326031685, -0.020025471225380898, 0.016044924035668373, -0.06299179047346115, -0.020097075030207634, 0.01829833723604679, 0.003226157743483782, 0.051708195358514786, -0.0038646955508738756, 0.04966

In [7]:
#LLM 답변 출력
from pprint import pprint
pprint(answer['result'])

("영화 'Up'는 2009년 5월 13일에 개봉했으며, 칼 프레드릭슨이라는 78세 노인이 전 세계를 탐험하고 인생을 최대한 경험하겠다는 "
 '평생의 꿈을 이루는 이야기를 담고 있습니다. 그의 8살짜리 꼬마 모험가 러셀과의 만남이 그의 삶에 새로운 전환점을 가져다줍니다. 이 '
 '영화의 평점은 7.7점이며, 상영 시간은 96분입니다.')


In [8]:
#특정 감독의 영화 찾기
answer = cypher_chain.invoke({"query": "'Christopher Nolan' 감독의 영화를 모두 찾아주세요."})

print(answer)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (:Person {name: "Christopher Nolan"})-[:DIRECTED]->(m:Movie) RETURN m
Full Context:
[{'m': {'overview': "Suffering short-term memory loss after a head injury, Leonard Shelby embarks on a grim quest to find the lowlife who murdered his wife in this gritty, complex thriller that packs more knots than a hangman's noose. To carry out his plan, Shelby snaps Polaroids of people and places, jotting down contextual notes on the backs of photos to aid in his search and jog his memory. He even tattoos his own body in a desperate bid to remember.", 'rating': 8.1, 'runtime': 113, 'tagline': 'Some memories are best forgotten.', 'content_embedding': [0.03619104623794556, -0.031051399186253548, -0.021772386506199837, -0.010778607800602913, 0.0340142585337162, 0.00996983703225851, 0.03650622442364693, 0.04481440782546997, -0.011185172945261002, 0.046710189431905746, 0.021561963483691216, -0.0398014560341835, 0.11584722250699997, 0.02

In [9]:
## LLM 답변 출력
pprint(answer['result'])

('Christopher Nolan 감독의 영화는 다음과 같습니다:\n'
 '* Memento\n'
 '* Insomnia\n'
 '* Batman Begins\n'
 '* The Prestige\n'
 '* The Dark Knight\n'
 '* Inception\n'
 '* The Dark Knight Rises\n'
 '* Interstellar')


In [10]:
#장르별 영화 검색
answer = cypher_chain.invoke({"query": "2010년 이후 개봉한 'Action' 장르 영화 중 평점이 높은 순서로 5개를 보여주세요."})

print(answer)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)
WHERE g.name = 'Action' AND m.released >= '2010-01-01'
RETURN m.title, m.released, m.rating
ORDER BY m.rating DESC
LIMIT 5

Full Context:
[{'m.title': 'Inception', 'm.released': '2010-07-14', 'm.rating': 8.1}, {'m.title': 'Batman: The Dark Knight Returns, Part 2', 'm.released': '2013-01-18', 'm.rating': 7.9}, {'m.title': 'Guardians of the Galaxy', 'm.released': '2014-07-30', 'm.rating': 7.9}, {'m.title': 'Big Hero 6', 'm.released': '2014-10-24', 'm.rating': 7.8}, {'m.title': 'Warrior', 'm.released': '2011-09-09', 'm.rating': 7.7}]

> Finished chain.
{'query': "2010년 이후 개봉한 'Action' 장르 영화 중 평점이 높은 순서로 5개를 보여주세요.", 'result': "'Inception' (2010) - 평점 8.1"}


In [11]:
pprint(answer)

{'query': "2010년 이후 개봉한 'Action' 장르 영화 중 평점이 높은 순서로 5개를 보여주세요.",
 'result': "'Inception' (2010) - 평점 8.1"}


#### 3) 출력 갯수를 지정 (top k)

In [12]:
#top_k 파라미터로 결과 수 제한
cypher_chain = GraphCypherQAChain.from_llm(
    llm = llm,
    graph= graph,
    allow_dangerous_requests=True,
    verbose=True,
    top_k = 5, # 최대 5개의 결과만 반환
)

answer = cypher_chain.invoke({"query":"'Tom Hanks' 배우가 출연한 영화를 모두 알려주세요."})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN m.title
Full Context:
[{'m.title': 'Splash'}, {'m.title': 'Big'}, {'m.title': 'A League of Their Own'}, {'m.title': 'Philadelphia'}, {'m.title': 'Forrest Gump'}]

> Finished chain.


In [13]:
pprint(answer['result'])

"'Splash', 'Big', 'A League of Their Own', 'Philadelphia', 'Forrest Gump'"


#### 4) 중간 과정 확인

In [14]:
# 중간 결과를 포함하여 출력
cypher_chain = GraphCypherQAChain.from_llm(
    llm = llm,
    graph= graph,
    allow_dangerous_requests=True,
    verbose=True,
    return_intermediate_steps=True # 생성된 Cypher 쿼리와 중간결과 확인
)

answer = cypher_chain.invoke({"query":"평점이 8점 이상인 'Drama' 장르 영화는 무엇이 있나요?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)
WHERE g.name = 'Drama' AND m.rating >= 8.0
RETURN m.title

Full Context:
[{'m.title': 'Metropolis'}, {'m.title': 'Modern Times'}, {'m.title': 'The Prisoner of Zenda'}, {'m.title': "It's a Wonderful Life"}, {'m.title': 'Seven Samurai'}, {'m.title': 'On the Waterfront'}, {'m.title': '12 Angry Men'}, {'m.title': 'The Apartment'}, {'m.title': 'Psycho'}, {'m.title': 'To Kill a Mockingbird'}]

> Finished chain.


In [15]:
#중간 결과 확인
for k, v in answer.items():
    print(f"{k}: {v}")

query: 평점이 8점 이상인 'Drama' 장르 영화는 무엇이 있나요?
result: 저는 평점이 8점 이상인 'Drama' 장르 영화에 대한 정보가 없습니다.
intermediate_steps: [{'query': "cypher\nMATCH (m:Movie)-[:IN_GENRE]->(g:Genre)\nWHERE g.name = 'Drama' AND m.rating >= 8.0\nRETURN m.title\n"}, {'context': [{'m.title': 'Metropolis'}, {'m.title': 'Modern Times'}, {'m.title': 'The Prisoner of Zenda'}, {'m.title': "It's a Wonderful Life"}, {'m.title': 'Seven Samurai'}, {'m.title': 'On the Waterfront'}, {'m.title': '12 Angry Men'}, {'m.title': 'The Apartment'}, {'m.title': 'Psycho'}, {'m.title': 'To Kill a Mockingbird'}]}]


#### 5) 직접 Cypher 결과 얻기 (LLM 답변 없이)

In [19]:
#직접 cypher결과 얻기 (LLM 답변 없이)
cypher_chain = GraphCypherQAChain.from_llm(
    llm=llm,
    graph=graph,
    allow_dangerous_requests=True,
    verbose=True,
    return_direct=True #LLM 답변 생성 단계 건너뛰기
)

answer = cypher_chain.invoke({"query":"2000년대(2000~2009) 개봉한 영화를 평점 순으로 정렬해주세요."})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie)
WHERE m.released >= "2000-01-01" AND m.released <= "2009-12-31"
RETURN m
ORDER BY m.rating DESC


> Finished chain.


In [20]:
for k,v in answer.items():
    print(f"{k}: {v}")

query: 2000년대(2000~2009) 개봉한 영화를 평점 순으로 정렬해주세요.
result: [{'m': {'overview': 'An aging out of work clown returns to his small hometown, resigned to spend the rest of his days in a drunken stupor. But when his passion for clowning is reawakened by the local amateur circus he finds his smile.', 'rating': 10.0, 'runtime': 0, 'content_embedding': [-0.00556557672098279, -0.0033987918868660927, 0.004887751769274473, 0.006810761522501707, 0.04638969898223877, 0.0009882241720333695, 0.039907317608594894, 0.01139283087104559, -0.0004392100381664932, 0.01404537446796894, -0.009779122658073902, -0.020709047093987465, 0.06199235841631889, 0.04058293253183365, 0.051371585577726364, 0.008947925642132759, -0.05487747862935066, 0.03452606126666069, -0.04968560114502907, -0.020361460745334625, 0.03883122280240059, 0.016740020364522934, 0.02850416675209999, 0.026502707973122597, -0.006584439892321825, 0.027189351618289948, -0.029451649636030197, -0.017331942915916443, -0.028499793261289597, -0.0220852922

In [21]:
import pandas as pd
pd.DataFrame([item['m'] for item in answer['result']])

,overview,rating,runtime,content_embedding,id,title,released,tagline
0,An aging out of work clown returns to his smal...,10.0,0,"[-0.00556557672098279, -0.0033987918868660927,...",movie-4662,Little Big Top,2006-01-01,NaN
1,A ten year old girl who wanders away from her ...,8.3,125,"[-0.021199902519583702, 0.0024180677719414234,...",movie-2294,Spirited Away,2001-07-20,The tunnel led Chihiro to a mysterious town...
2,A word for word depiction of the life of Jesus...,8.2,125,"[0.0018628669204190373, -0.014324923977255821,...",movie-2947,The Visual Bible: The Gospel of John,2003-09-11,For God loved the world So much...
3,"When Sophie, a shy young woman, is cursed with...",8.2,119,"[-0.03511136397719383, 0.021571852266788483, -...",movie-1987,Howl's Moving Castle,2004-11-19,The two lived there
4,Batman raises the stakes in his war on crime. ...,8.2,152,"[-0.0036215619184076786, -0.025119144469499588...",movie-65,The Dark Knight,2008-07-16,Why So Serious?
5,Suffering short-term memory loss after a head ...,8.1,113,"[0.03619104623794556, -0.031051399186253548, -...",movie-3573,Memento,2000-10-11,Some memories are best forgotten.
6,Cidade de Deus is a shantytown that started du...,8.1,130,"[-0.026186905801296234, -0.006241500843316317,...",movie-3866,City of God,2002-02-05,"If you run you're dead... if you stay, you're ..."
7,Aragorn is revealed as the heir to the ancient...,8.1,201,"[-0.05729801207780838, 0.01947001740336418, 0....",movie-329,The Lord of the Rings: The Return of the King,2003-12-01,The eye of the enemy is moving.
8,"Young hobbit Frodo Baggins, after inheriting a...",8.0,178,"[-0.04288017377257347, 0.03473649546504021, 0....",movie-262,The Lord of the Rings: The Fellowship of the Ring,2001-12-18,One ring to rule them all
9,"Seymour Polatkin is a successful, gay Indian p...",8.0,103,"[0.014240596443414688, -0.018224861472845078, ...",movie-4678,The Business of Fancydancing,2002-01-14,Sometimes going home is the hardest journey of...


In [23]:
#Cypher쿼리 생성 모델과 최종답변 생성 모델을 따로 사용하고 싶은 경우
cypher_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite",temperature=0)
qa_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite",temperature=0)

cypher_chain=GraphCypherQAChain.from_llm(
    cypher_llm = cypher_llm,
    qa_llm = qa_llm,
    graph=graph,
    allow_dangerous_requests=True,
    verbose=True,
)

answer = cypher_chain.invoke({"query":"배우 'Leonardo DiCaprio'와 감독 'Christopher Nolan'이 함꼐 작업한 영화가 있나요?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p1:Person {name: 'Leonardo DiCaprio'})-[:ACTED_IN]->(m:Movie)<-[:DIRECTED]-(p2:Person {name: 'Christopher Nolan'})
RETURN m.title

Full Context:
[{'m.title': 'Inception'}]

> Finished chain.


In [24]:
for k,v in answer.items():
    print(f"{k}: {v}")

query: 배우 'Leonardo DiCaprio'와 감독 'Christopher Nolan'이 함꼐 작업한 영화가 있나요?
result: 네, 'Inception'이라는 영화가 있습니다.


#### 7) 커스텀 프롬프트 활용

In [25]:
from langchain_core.prompts.prompt import PromptTemplate

#Cypher 생성을 위한 영화 데이터베이스 특화 프롬프트
CYPHER_GENERATION_TEMPLATE="""Task: Generate Cypher statement to question a movie graph database.
Instructions : 
- Use only the provided node labels, relationship types, and properties in the schema.
- Do not use any relationship types or properties not specified in the schema.
- Focus on extracting meaningful insights from movie data.

Schema:
{schema}

Note:
- Provide only the Cypher statement.
- Do not include explanations or aplolgies
- Generate precise, relevant Cypher queries

Examples:
# 특정 배우가 출연한 영화 찾기
MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name = 'Tom Hanks'
RETURN m.title, m.released, m.rating
ORDER BY m.released DESC

#특정 장르의 평점 높은 영화 찾기
MATCH (m:Movie)-[:IN_GENRE]->(g:Genre)
WHERE g.name = 'Action'
RETURn m.title, m.rating
ORDER BY m.rating DESC
LIMIT 5

#특정 연도에 개봉한 영화 조회
MATCH (m:Movie)
WHERE m.released = 2000
RETURN m.title, m.rating
ORDER BY m.rating DESC

#특정 배우와 감독이 함께 작업한 영화 찾기
MATCH (actor:Person)-[:ACTED_IN]->(m:Movie)<-[:DIRECTED]-(director:Person)
WHERE actor.name = 'Leonardo DiCaprio' AND director.name = 'Christopher Nolan'
RETURN m.title, m.released, m.rating

The question is:
{question}
"""

#결과 처리를 위한 영화 QA 프롬프트
QA_TEMPLATE = """
당신은 영화 데이터베이스 분석 전문가로, 영화 정보에 대한 명확하고 간결한 정보를 한국어로 제공합니다.

[질문]
{question}

[검색 결과]
{context}

#응답 가이드라인 :
- 검색 결과에서 핵심 정보를 요약하세요
- 영화 데이터에 대한 명확하고 객관적인 개요를 제공하세요
- 전문적이고 유익한 톤을 사용하세요
- 영화 데이터에서 중요한 패턴이나 트렌드를 강조하세요
- 맥락이 불충분한 경우 더 많은 정보가 필요하다고 명확히 언급하세요
- 추측이나 개인적인 해석은 피하세요

#응답형식:
- 간략한 발견 요약으로 시작하세요.
- 여러 영화가 발견된 경우 간결한 개요를 제공하세요
- 가독성을 위해 글머리 기호나 짧은 단락을 사용하세요
- 개봉일, 평점, 주요 배우나 감독과 같은 관련 세부 정보를 포함하세요
- 모든 숫자 데이터나 기술 용어를 이해하기 쉬운 언어로 번역하세요

#예시 응답 구조:
"분석 결과, [주요 발견 요약]

주요 특징:
- [첫 번째 중요 인사이트]
- [두 번째 중요 인사이트]

추가정보: [필요한 경우 추가 설명]"
"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"],
    template = CYPHER_GENERATION_TEMPLATE
)

QA_PROMPT = PromptTemplate(
    input_variables=["question","context"],
    template = QA_TEMPLATE
)

#Chain 생성 - 주목 : input_key와 output_key를 명시적으로 설정
cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm = cypher_llm,
    qa_llm = qa_llm,
    graph = graph,
    allow_dangerous_requests = True,
    verbose = True,
    cypher_prompt = CYPHER_GENERATION_PROMPT,
    qa_prompt=QA_PROMPT,
    input_key="question",
    output_key="result"
)

#Cypher 쿼리 실행
answer = cypher_chain.invoke({"question":"배우 'Leonardo DiCaprio'와 감독 'Christopher Nolan'이 함께 작업한 영화가 있나요?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (actor:Person)-[:ACTED_IN]->(m:Movie)<-[:DIRECTED]-(director:Person)
WHERE actor.name = 'Leonardo DiCaprio' AND director.name = 'Christopher Nolan'
RETURN m.title, m.released, m.rating
Full Context:
[{'m.title': 'Inception', 'm.released': '2010-07-14', 'm.rating': 8.1}]

> Finished chain.


In [26]:
for k,v in answer.items():
    print(f"{k}: {v}")

question: 배우 'Leonardo DiCaprio'와 감독 'Christopher Nolan'이 함께 작업한 영화가 있나요?
result: 분석 결과, 배우 'Leonardo DiCaprio'와 감독 'Christopher Nolan'이 함께 작업한 영화로 'Inception'이 확인되었습니다.

주요 특징:
*   **영화 제목:** Inception
*   **개봉일:** 2010년 7월 14일
*   **평점:** 8.1

이 영화는 두 거장의 협업을 통해 탄생했으며, 높은 평점을 기록하며 작품성을 인정받았습니다.
